In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_3.csv")

In [3]:
df

,timestamp_local,temp,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,430,1782.5,60.7,2.7,533.7,377.00,253.3
...,...,...,...,...,...,...,...,...,...
20973,23/06/2024 19:00,39.3,92,155.6,21.0,105.3,57.0,31.33,17.0
20974,23/06/2024 20:00,37.0,98,191.4,30.0,80.0,58.0,34.00,20.0
20975,23/06/2024 21:00,35.6,98,233.6,39.7,60.7,58.0,34.33,23.7
20976,23/06/2024 22:00,34.4,99,275.8,49.3,41.3,58.0,34.67,27.3


In [4]:
data = df[['o3']]

In [5]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [6]:
scaled = scaled.reshape((20978,))

In [7]:
scaled.shape

(20978,)

In [8]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [9]:
def train_test_split(X, y, test_size=0.2):
    split_idx = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    return X_train, X_test, y_train, y_test

In [10]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

X_max = X_train.max()
X_min = X_train.min()

In [11]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [12]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 32)             │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 5312)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         892,584 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 892,712 (3.41 MB)

 Trainable params: 892,712 (3.41 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0120 - root_mean_squared_error: 0.1064 - val_loss: 0.0047 - val_root_mean_squared_error: 0.0687 - learning_rate: 0.0010
Epoch 2/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0069 - root_mean_squared_error: 0.0833 - val_loss: 0.0043 - val_root_mean_squared_error: 0.0659 - learning_rate: 0.0010
Epoch 3/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0065 - root_mean_squared_error: 0.0809 - val_loss: 0.0043 - val_root_mean_squared_error: 0.0657 - learning_rate: 0.0010
Epoch 4/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0064 - root_mean_squared_error: 0.0797 - val_loss: 0.0042 - val_root_mean_squared_error: 0.0651 - learning_rate: 0.0010
Epoch 5/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0062 - root_mean_squared_error: 0.0789 - val_loss: 0.0044 - val_root_mean_squared_error: 0.0664 - learning_rate: 0.0010
Epoch 6/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0062 - root_mean_squared_er

In [13]:
forecast = model.predict(X_test)

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [14]:
poll = np.array(df["o3"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 3.6203837


In [17]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 2.983742579817772


In [18]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 97.01625742018223 %


In [19]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 2.8091164


In [20]:
y_test_true.shape

(4129, 168)

In [21]:
testPredict.shape

(4129, 168)

In [22]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [23]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 1.0215
RMSE for segment 2: 1.6904
RMSE for segment 3: 2.2433
RMSE for segment 4: 2.6067
RMSE for segment 5: 2.8543
RMSE for segment 6: 3.0095
RMSE for segment 7: 3.0986
RMSE for segment 8: 3.1453
RMSE for segment 9: 3.1610
RMSE for segment 10: 3.1651
RMSE for segment 11: 3.1657
RMSE for segment 12: 3.1673
RMSE for segment 13: 3.1672
RMSE for segment 14: 3.1645
RMSE for segment 15: 3.1646
RMSE for segment 16: 3.1625
RMSE for segment 17: 3.1584
RMSE for segment 18: 3.1550
RMSE for segment 19: 3.1566
RMSE for segment 20: 3.1572
RMSE for segment 21: 3.1586
RMSE for segment 22: 3.1636
RMSE for segment 23: 3.1731
RMSE for segment 24: 3.1844
RMSE for segment 25: 3.2388
RMSE for segment 26: 3.3118
RMSE for segment 27: 3.3880
RMSE for segment 28: 3.4509
RMSE for segment 29: 3.4922
RMSE for segment 30: 3.5276
RMSE for segment 31: 3.5467
RMSE for segment 32: 3.5585
RMSE for segment 33: 3.5654
RMSE for segment 34: 3.5637
RMSE for segment 35: 3.5614
RMSE for segment 36: 3.5575
R

In [24]:
forecast

array([[0.11756711, 0.11607878, 0.11496904, ..., 0.12688452, 0.09779279,
        0.08172522],
       [0.11848933, 0.11561473, 0.11464678, ..., 0.09680837, 0.08141424,
        0.06115625],
       [0.11369924, 0.11017311, 0.11323448, ..., 0.07453654, 0.05615649,
        0.05629455],
       ...,
       [0.26169467, 0.22674862, 0.18718007, ..., 0.39512214, 0.33219934,
        0.2713744 ],
       [0.22765294, 0.18851432, 0.16719136, ..., 0.3287981 , 0.2718113 ,
        0.23627317],
       [0.19199616, 0.17127296, 0.14936413, ..., 0.2694223 , 0.23491503,
        0.19770367]], dtype=float32)

In [25]:
testPredict

array([[ 86.12193 ,  86.04076 ,  85.980225, ...,  86.63013 ,  85.0434  ,
         84.16704 ],
       [ 86.17223 ,  86.01544 ,  85.962654, ...,  84.98971 ,  84.15008 ,
         83.045166],
       [ 85.91097 ,  85.71865 ,  85.88562 , ...,  83.774956,  82.77247 ,
         82.78    ],
       ...,
       [ 93.98295 ,  92.07692 ,  89.91877 , ..., 101.26036 ,  97.828415,
         94.5109  ],
       [ 92.12624 ,  89.99154 ,  88.82854 , ...,  97.64291 ,  94.53473 ,
         92.596405],
       [ 90.18144 ,  89.05116 ,  87.85621 , ...,  94.40443 ,  92.52233 ,
         90.492744]], dtype=float32)

In [26]:
y_test_true

array([[ 85.98279 ,  85.98279 ,  85.98279 , ...,  87.899605,  86.67981 ,
         85.46002 ],
       [ 85.98279 ,  85.98279 ,  85.98279 , ...,  86.67981 ,  85.46002 ,
         84.24023 ],
       [ 85.98279 ,  85.98279 ,  84.81528 , ...,  85.46002 ,  84.24023 ,
         84.64102 ],
       ...,
       [ 93.87657 ,  92.377975,  90.86195 , ..., 102.48481 ,  98.058716,
         93.65004 ],
       [ 92.377975,  90.86195 ,  88.6489  , ...,  98.058716,  93.65004 ,
         90.2869  ],
       [ 90.86195 ,  88.6489  ,  86.45328 , ...,  93.65004 ,  90.2869  ,
         86.90634 ]], dtype=float32)

In [27]:
model.save("o3_1dcnn.keras")